<h1> <center>
🐀🐀🐀🐀🐀🐀🐀🐀🐀🐀🐀🐁🐁🐁🐁🐁🐁🐁🐁🐁🐁<br>
🐀 MABe Classical Classification: Baseline 💪 🐁<br>
🐀🐀🐀🐀🐀🐀🐀🐀🐀🐀🐀🐁🐁🐁🐁🐁🐁🐁🐁🐁🐁
</center>
</h1>

<p align="center"><img src="https://images.aicrowd.com/uploads/ckeditor/pictures/324/content_task1_structure.png">
</p>

In [1]:
#Select GPU
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
cd /home/blansdel/projects/mabe-task1/

/home/blansdel/projects/mabe-task1


In [3]:
#submission
%connect_info

{
  "shell_port": 36731,
  "iopub_port": 44915,
  "stdin_port": 40337,
  "control_port": 34757,
  "hb_port": 44493,
  "ip": "127.0.0.1",
  "key": "d919f831-3b5e62580b9149f8eee826b0",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-c0ed0a48-69c1-4e45-923f-a6d76a0845cf.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


# Import necessary modules and packages 📚


In [4]:
import numpy as np
import os

from tensorflow import keras
import tensorflow as tf
from keras.models import Sequential
import keras.layers as layers
import tensorflow_addons as tfa

import sklearn
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy
import tqdm
import gc

from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, LeaveOneGroupOut
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score, f1_score, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from lib.utils import compute_metrics, print_metrics, validate_submission

# Download the dataset 📲

Please get your API key from https://www.aicrowd.com/participants/me


In [5]:
API_KEY = "0ba231d61506b40a4ae00df011cf0cb9"
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


# Load Data
The dataset files are python dictionaries, [this](https://colab.research.google.com/drive/1ddCX-TAdEcsUaGf09f5Glgr_G57FMK_O#scrollTo=JPsfxdl2GMcM&line=18&uniqifier=1) is a descirption of how the data is organized.



In [6]:
train = np.load('data/train.npy',allow_pickle=True).item()
test = np.load('data/test.npy',allow_pickle=True).item()
sample_submission = np.load('data/sample_submission.npy',allow_pickle=True).item()

# Training The Model 🏋️‍♂️

The given MABe dataset contain many sequences of time series data, each frame has its own behavior label. Training on just a single frame does not give good results due to less information. 

So here past and future frames are also added to each input. But also all the frames are not concatenated as as the boundaries of the past and future frames need to stay separate for each video.


## Seeding helper
Its good practice to seed before every run, that way its easily reproduced.

In [7]:
def seed_everything(seed):
  np.random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  tf.random.set_seed(seed)

seed=2021
seed_everything(seed)

# Note: Image processing may be slow if too many frames are animated.                
 
#Plotting constants
FRAME_WIDTH_TOP = 1024
FRAME_HEIGHT_TOP = 570
 
RESIDENT_COLOR = 'lawngreen'
INTRUDER_COLOR = 'skyblue'
 
PLOT_MOUSE_START_END = [(0, 1), (0, 2), (1, 3), (2, 3), (3, 4),
                        (3, 5), (4, 6), (5, 6), (1, 2)]
 
class_to_color = {'other': 'white', 'attack' : 'red', 'mount' : 'green',
                  'investigation': 'orange'}
 
class_to_number = {s: i for i, s in enumerate(train['vocabulary'])}
 
number_to_class = {i: s for i, s in enumerate(train['vocabulary'])}
 
def num_to_text(anno_list):
  return np.vectorize(number_to_class.get)(anno_list)

## Generator 🔌

The generator is used to take input winodws from each sequence after randomly sampling frames. 

It also provides code for augmentations
1.   Random rotation
2.   Random translate

🚧 Note that these augmentations are applied in the same across all frames in a selected window, e.g - Random rotation by 10 degrees will rotate all frames in the input window by the same angle.




In [8]:
class MABe_Generator(keras.utils.Sequence):
    def __init__(self, pose_dict, 
                 batch_size, dim, 
                 use_conv, num_classes, augment=False,
                 class_to_number=None,
                 past_frames=0, future_frames=0, 
                 frame_gap=1, shuffle=False,
                 mode='fit'):
        self.batch_size = batch_size
        self.video_keys = list(pose_dict.keys())
        self.dim = dim
        self.use_conv = use_conv
        self.past_frames = past_frames
        self.future_frames = future_frames
        self.frame_gap = frame_gap
        self.shuffle = shuffle
        self.num_classes=num_classes
        self.augment = augment
        self.mode = mode

        self.class_to_number = class_to_number

        self.video_indexes = []
        self.frame_indexes = []
        self.X = {}
        if self.mode == 'fit':
          self.y = []
        self.pad = self.past_frames * self.frame_gap
        future_pad = self.future_frames * self.frame_gap
        pad_width = (self.pad, future_pad), (0, 0), (0, 0), (0, 0)
        self.seq_lengths = {}
        for vc, key in enumerate(self.video_keys):
          if self.mode == 'fit':
            anno = pose_dict[key]['annotations']
            self.y.extend(anno)
          nframes = len(pose_dict[key]['keypoints'])
          self.video_indexes.extend([vc for _ in range(nframes)])
          self.frame_indexes.extend(range(nframes))
          self.X[key] = np.pad(pose_dict[key]['keypoints'], pad_width)
          self.seq_lengths[key] = nframes
        
        if self.mode == 'fit':
          self.y = np.array(self.y)
        
        self.X_dtype = self.X[key].dtype

        self.indexes = list(range(len(self.frame_indexes)))

        if self.mode == 'predict':
          extra_predicts = -len(self.indexes) % self.batch_size # So that last part is not missed
          self.indexes.extend(self.indexes[:extra_predicts])
          self.indexes = np.array(self.indexes)
        
        self.on_epoch_end()

    def __len__(self):
        return len(self.indexes) // self.batch_size

    def augment_fn(self, x):
      # Rotate
      angle = (np.random.rand()-0.5) * (np.pi * 2)
      c, s = np.cos(angle), np.sin(angle)
      rot = np.array([[c, -s], [s, c]])
      x = np.dot(x, rot)

      # Shift - All get shifted together
      shift = (np.random.rand(2)-0.5) * 2 * 0.25
      x = x + shift
      return x

    def __getitem__(self, index):
        bs = self.batch_size
        indexes = self.indexes[index*bs:(index+1)*bs]
        X = np.empty((bs, *self.dim), self.X_dtype)
        if self.mode == 'predict':
          vkey_fi_list = []
        for bi, idx in enumerate(indexes):
          vkey = self.video_keys[self.video_indexes[idx]]
          fi = self.frame_indexes[idx]
          if self.mode == 'predict':
            vkey_fi_list.append((vkey, fi))
          fi = fi + self.pad
          start = fi - self.past_frames*self.frame_gap
          stop = fi + (self.future_frames + 1)*self.frame_gap
          assert start >= 0

          Xi = self.X[vkey][start:stop:self.frame_gap].copy()
          
          if self.augment:
            Xi = self.augment_fn(Xi)
          X[bi] = np.reshape(Xi, self.dim)
          

        if self.mode == 'fit':
          y_vals = self.y[indexes]
          # Converting to one hot because F1 callback needs one hot
          y = np.zeros( (bs,self.num_classes), np.float32)
          y[np.arange(bs), y_vals] = 1
          return X, y

        elif self.mode == 'predict':
          return X, vkey_fi_list

    def on_epoch_end(self):
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

## Trainer 🏋️

The trainer class implements a unified interface for using the datagenerator.

It supports fully connected or 1D convolutional networks, as well as other hyperparameters for the model and the generator.

In [75]:
class Trainer:
  def __init__(self, *,
               train_data,
               val_data,
               test_data,
               feature_dim, 
               batch_size, 
               num_classes,
               stacking_data = None,
               augment=False,
               class_to_number=None,
               past_frames=0, 
               future_frames=0,
               frame_gap=1, 
               use_conv=False):
    flat_dim = np.prod(feature_dim)
    if use_conv:
      input_dim = ((past_frames + future_frames + 1), flat_dim,)
    else:
      input_dim = (flat_dim * (past_frames + future_frames + 1),)

    self.input_dim = input_dim
    self.use_conv=use_conv
    self.num_classes=num_classes

    c2n = {'other': 0,'investigation': 1,
                'attack' : 2, 'mount' : 3}
    self.class_to_number = class_to_number or c2n

    self.train_generator = MABe_Generator(train_data, 
                                      batch_size=batch_size, 
                                      dim=input_dim,
                                      num_classes=num_classes, 
                                      past_frames=past_frames, 
                                      future_frames=future_frames,
                                      class_to_number=self.class_to_number,
                                      use_conv=use_conv,
                                      frame_gap=frame_gap,
                                      augment=augment,
                                      shuffle=True,
                                      mode='fit')

    self.val_generator = MABe_Generator(val_data, 
                                        batch_size=batch_size, 
                                        dim=input_dim, 
                                        num_classes=num_classes, 
                                        past_frames=past_frames,
                                        future_frames=future_frames,
                                        use_conv=use_conv,
                                        class_to_number=self.class_to_number,
                                        frame_gap=frame_gap,
                                        augment=False,
                                        shuffle=False,
                                        mode='fit')
    
    if stacking_data is not None:
        self.stacking_train_generator = MABe_Generator(stacking_data, 
                                            batch_size=batch_size, 
                                            dim=input_dim, 
                                            num_classes=num_classes, 
                                            past_frames=past_frames,
                                            future_frames=future_frames,
                                            use_conv=use_conv,
                                            class_to_number=self.class_to_number,
                                            frame_gap=frame_gap,
                                            augment=False,
                                            shuffle=False,
                                            mode='fit')        
    
    if test_data is not None:
      self.test_generator = MABe_Generator(test_data, 
                                        batch_size=8192, 
                                        dim=input_dim, 
                                        num_classes=num_classes, 
                                        past_frames=past_frames,
                                        future_frames=future_frames,
                                        use_conv=use_conv,
                                        class_to_number=self.class_to_number,
                                        frame_gap=frame_gap,
                                        augment=False,
                                        shuffle=False,
                                        mode='predict')
  
  def delete_model(self):
    self.model = None
  
  def initialize_model(self, layer_channels=(512, 256), dropout_rate=0., 
                       learning_rate=1e-3, conv_size=5):

    def add_dense_bn_activate(model, out_dim, activation='relu', drop=0.):
      model.add(layers.Dense(out_dim))
      model.add(layers.BatchNormalization())
      model.add(layers.Activation('relu'))
      if drop > 0:
        model.add(layers.Dropout(rate=drop))
      return model
    
    def add_conv_bn_activate(model, out_dim, activation='relu', conv_size=3, drop=0.):
      model.add(layers.Conv1D(out_dim, conv_size))
      model.add(layers.BatchNormalization())
      model.add(layers.Activation('relu'))
      model.add(layers.MaxPooling1D(2, 2))
      if drop > 0:
        model.add(layers.Dropout(rate=drop))
      return model

    model = Sequential()
    model.add(layers.Input(self.input_dim))
    model.add(layers.BatchNormalization())
    for ch in layer_channels:
      if self.use_conv:
        model = add_conv_bn_activate(model, ch, conv_size=conv_size,
                                     drop=dropout_rate)
      else:
        model = add_dense_bn_activate(model, ch, drop=dropout_rate)
    model.add(layers.Flatten())
    model.add(layers.Dense(self.num_classes, activation='softmax'))

    metrics = [tfa.metrics.F1Score(num_classes=self.num_classes)]
    optimizer = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=optimizer,
                  metrics=metrics)

    self.model = model

  def _set_model(self, model):
      """ Set an external, provide initialized and compiled keras model """
      self.model = model

  def train(self, epochs=20, class_weight=None):
    if self.model is None:
      print("Please Call trainer.initialize_model first")
      return
    self.model.fit(self.train_generator,
          validation_data=self.val_generator,
          epochs=epochs,
          class_weight=class_weight)
        
  def get_validation_labels(self, on_test_set=False):
    y_val = []
    for _, y in self.val_generator:
      y_val.extend(list(y))
    y_val = np.argmax(np.array(y_val), axis=-1)
    return y_val

  def get_training_pred_proba(self):
    y_val_pred = self.model.predict(self.train_generator)
    return y_val_pred

  def get_validation_pred_proba(self):
    y_val_pred = self.model.predict(self.val_generator)
    return y_val_pred

  def get_validation_predictions(self):
    y_val_pred = self.model.predict(self.val_generator)
    y_val_pred = np.argmax(y_val_pred, axis=-1)
    return y_val_pred

  def get_stacking_trainer_pred_proba(self):
    y_val_pred = self.model.predict(self.stacking_train_generator)
    return y_val_pred

  def get_trainer_predictions(self):
    y_pred = self.model.predict(self.train_generator)
    y_pred = np.argmax(y_pred, axis=-1)
    return y_pred

  def get_stacking_trainer_predictions(self):
    y_val_pred = self.model.predict(self.stacking_train_generator)
    y_val_pred = np.argmax(y_val_pred, axis=-1)
    return y_val_pred

  def get_validation_metrics(self):
    y_val = self.get_validation_labels()
    y_val_pred = self.get_validation_predictions()

    f1_scores = sklearn.metrics.f1_score(y_val, y_val_pred,average=None)
    rec_scores = sklearn.metrics.precision_score(y_val, y_val_pred,average=None)
    prec_scores = sklearn.metrics.recall_score(y_val, y_val_pred,average=None)
    classes = list(self.class_to_number.keys())
    metrics = pd.DataFrame({"Class": classes, "F1": f1_scores, "Precision": prec_scores, "Recall": rec_scores})
    return metrics

  def get_test_pred_proba(self):
    all_test_preds = {}
    for vkey in self.test_generator.video_keys:
      nframes = self.test_generator.seq_lengths[vkey]
      all_test_preds[vkey] = np.zeros((nframes, 4))

    for X, vkey_fi_list in tqdm.tqdm(self.test_generator):
      test_pred = np.array(self.model.predict(X))
      #test_pred = np.argmax(test_pred, axis=-1)

      for p, (vkey, fi) in zip(test_pred, vkey_fi_list):
        all_test_preds[vkey][fi] = p
    return all_test_preds

  def get_test_predictions(self):
    all_test_preds = {}
    for vkey in self.test_generator.video_keys:
      nframes = self.test_generator.seq_lengths[vkey]
      all_test_preds[vkey] = np.zeros(nframes, dtype=np.int32)

    for X, vkey_fi_list in tqdm.tqdm(self.test_generator):
      test_pred = self.model.predict(X)
      test_pred = np.argmax(test_pred, axis=-1)

      for p, (vkey, fi) in zip(test_pred, vkey_fi_list):
        all_test_preds[vkey][fi] = p
    return all_test_preds

## Preprocess

We'll normalize the data based on the information that the frame size is 1024x570

The original data is of shape (sequence length, mouse, x y coordinate, keypoint)
 = (length, 2, 2, 7)

 We'll swap the x y and the keypoint axis, which will help in rotation augmentation.


In [76]:
def normalize_data(orig_pose_dictionary):
  for key in orig_pose_dictionary:
    X = orig_pose_dictionary[key]['keypoints']
    X = X.transpose((0,1,3,2)) #last axis is x, y coordinates
    X[..., 0] = X[..., 0]/1024
    X[..., 1] = X[..., 1]/570
    orig_pose_dictionary[key]['keypoints'] = X
  return orig_pose_dictionary

## Dataset split
Since MABe has multiple sequences, it is sensible to split the dataset based on different sequences rather than randomly sampling frames, which may leak information.

About half the sequences don't have "attack" behavior, hence we'll stratify based on whether "attack" behavior is present or absent.

This function only does a single split, but you can also do multiple splits for cross validation.

For Task 2 and 3 there are very few sequences, hence we split the sequences in half for validation.

In [77]:
def split_validation(orig_pose_dictionary, vocabulary, seed=2021, 
                       test_size=0.5, split_videos=False):
  if split_videos:
    pose_dictionary = {}
    for key in orig_pose_dictionary:
      key_pt1 = key + '_part1'
      key_pt2 = key + '_part2'
      anno_len = len(orig_pose_dictionary[key]['annotations'])
      split_idx = anno_len//2
      pose_dictionary[key_pt1] = {
          'annotations': orig_pose_dictionary[key]['annotations'][:split_idx],
          'keypoints': orig_pose_dictionary[key]['keypoints'][:split_idx]}
      pose_dictionary[key_pt2] = {
          'annotations': orig_pose_dictionary[key]['annotations'][split_idx:],
          'keypoints': orig_pose_dictionary[key]['keypoints'][split_idx:]}
  else:
    pose_dictionary = orig_pose_dictionary
  
  def get_percentage(sequence_key):
    anno_seq = num_to_text(pose_dictionary[sequence_key]['annotations'])
    counts = {k: np.mean(np.array(anno_seq) == k) for k in vocabulary}
    return counts

  anno_percentages = {k: get_percentage(k) for k in pose_dictionary}

  anno_perc_df = pd.DataFrame(anno_percentages).T

  rng_state = np.random.RandomState(seed)
  try:
    idx_train, idx_val = train_test_split(anno_perc_df.index,
                                      stratify=anno_perc_df['attack'] > 0, 
                                      test_size=test_size,
                                      random_state=rng_state)
  except:
    idx_train, idx_val = train_test_split(anno_perc_df.index,
                                      test_size=test_size,
                                      random_state=rng_state)
    
  train_data = {k : pose_dictionary[k] for k in idx_train}
  val_data = {k : pose_dictionary[k] for k in idx_val}
  return train_data, val_data, anno_perc_df

In [78]:
#This version makes two validation datasets... one for fitting the stacked estimator
#and another for computing test performance

#Removes support for split videos
def split_validation_two(orig_pose_dictionary, vocabulary, seed=2021, 
                       test_size = 0.2, split_videos=False, stacking_size = 0.2):
    
  total_size = min(test_size + stacking_size, 0.5)

  pose_dictionary = orig_pose_dictionary
  
  def get_percentage(sequence_key):
    anno_seq = num_to_text(pose_dictionary[sequence_key]['annotations'])
    counts = {k: np.mean(np.array(anno_seq) == k) for k in vocabulary}
    return counts

  anno_percentages = {k: get_percentage(k) for k in pose_dictionary}

  anno_perc_df = pd.DataFrame(anno_percentages).T

  rng_state = np.random.RandomState(seed)
  try:
    idx_train, idx = train_test_split(anno_perc_df.index,
                                      stratify=anno_perc_df['attack'] > 0, 
                                      test_size=total_size,
                                      random_state=rng_state)
    idx_test, idx_val = train_test_split(idx,
                                  test_size=stacking_size/total_size,
                                  random_state=rng_state)
    print("Successfully stratified training data")    
  except:
    idx_train, idx = train_test_split(anno_perc_df.index,
                                      test_size=total_size,
                                      random_state=rng_state)
    idx_test, idx_val = train_test_split(idx,
                                  test_size=stacking_size/total_size,
                                  random_state=rng_state)
    
  train_data = {k : pose_dictionary[k] for k in idx_train}
  val_data = {k : pose_dictionary[k] for k in idx_val}
  test_data = {k : pose_dictionary[k] for k in idx_test}
  return train_data, val_data, test_data, anno_perc_df

# Train function and inference

This below function is specific for Task 1, it has a set of hyperparameters we found with some tuning. Though results can be improved with further tuning.

It also generates the submission dictionary after training is completed.

### Get baseline trained

In [79]:
#Params
results_dir = './results_baseline_net/'
dataset = train['sequences']
vocabulary = train['vocabulary']
test_data = test['sequences']
seed = 2021
epochs = 50

augment = False
skip_test_prediction = False

stacking_size = 0.2

HPARAMS = {}
val_size = HPARAMS["val_size"] = 0.2
normalize = HPARAMS["normalize"] = True
HPARAMS["seed"] = seed
seed_everything(seed)
split_videos = HPARAMS["split_videos"] = False

# Generator parameters
past_frames = HPARAMS["past_frames"] = 50
future_frames = HPARAMS["future_frames"] = 50
frame_gap = HPARAMS["frame_gap"] = 1
use_conv = HPARAMS["use_conv"] = True
batch_size = HPARAMS["batch_size"] = 128

# Model parameters
dropout_rate = HPARAMS["dropout_rate"] = 0.5
learning_rate = HPARAMS["learning_rate"] = 5e-4
layer_channels = HPARAMS["layer_channels"] = (128, 64, 32)
conv_size = HPARAMS["conv_size"] = 5
augment = HPARAMS["augment"] = augment
class_to_number = HPARAMS['class_to_number'] = vocabulary
epochs = HPARAMS["epochs"] = epochs

In [80]:
if normalize:
    dataset = normalize_data(deepcopy(dataset))
if not skip_test_prediction:
    test_data = normalize_data(deepcopy(test_data))
else:
    test_data = None

train_data, val_data, stacking_training_data, anno_perc_df = split_validation_two(dataset, 
                                                    seed=seed,
                                                    vocabulary=vocabulary,
                                                    test_size=val_size, 
                                                    stacking_size = stacking_size,
                                                    split_videos=split_videos)                               
num_classes = len(anno_perc_df.keys())
feature_dim = HPARAMS["feature_dim"] = (2,7,2)

trainer = Trainer(train_data=train_data,
                val_data=val_data,
                test_data=test_data,
                stacking_data = stacking_training_data,
                feature_dim=feature_dim, 
                batch_size=batch_size, 
                num_classes=num_classes,
                augment=augment,
                class_to_number=class_to_number,
                past_frames=past_frames, 
                future_frames=future_frames,
                frame_gap=frame_gap,
                use_conv=use_conv)

trainer.initialize_model(layer_channels=layer_channels,
                      dropout_rate=dropout_rate,
                      learning_rate=learning_rate,
                      conv_size=conv_size)

Successfully stratified training data


In [81]:
trainer.train(epochs=epochs)
augment_str = '_augmented' if augment else ''
trainer.model.save(f'{results_dir}/task1{augment_str}.h5')
np.save(f"{results_dir}/task1{augment_str}_hparams", HPARAMS)

Epoch 1/50
2579/2579 [==============================] - 21s 8ms/step - loss: 0.6161 - f1_score: 0.4776 - val_loss: 0.4088 - val_f1_score: 0.5593
Epoch 2/50
2579/2579 [==============================] - 20s 8ms/step - loss: 0.3409 - f1_score: 0.7624 - val_loss: 0.3493 - val_f1_score: 0.6816
Epoch 3/50
2579/2579 [==============================] - 20s 8ms/step - loss: 0.3024 - f1_score: 0.8118 - val_loss: 0.3618 - val_f1_score: 0.6824
Epoch 4/50
2579/2579 [==============================] - 20s 8ms/step - loss: 0.2900 - f1_score: 0.8248 - val_loss: 0.3639 - val_f1_score: 0.6855
Epoch 5/50
2579/2579 [==============================] - 20s 8ms/step - loss: 0.2726 - f1_score: 0.8346 - val_loss: 0.3561 - val_f1_score: 0.6671
Epoch 6/50
2579/2579 [==============================] - 20s 8ms/step - loss: 0.2670 - f1_score: 0.8415 - val_loss: 0.3406 - val_f1_score: 0.7140
Epoch 7/50
2579/2579 [==============================] - 20s 8ms/step - loss: 0.2557 - f1_score: 0.8488 - val_loss: 0.3437 - val_f1

### Get predicted probabilities for input into stacked model


Validation and training data

In [15]:
y_val_pred_proba = trainer.get_validation_pred_proba()
y_train_pred_proba = trainer.get_training_pred_proba()
y_stacking_train_pred_proba = trainer.get_stacking_trainer_pred_proba()

In [16]:
y_val_pred = trainer.get_validation_predictions()
#y_train_pred = trainer.get_trainer_predictions()
y_stacking_train_pred = trainer.get_stacking_trainer_predictions()

### Prepare data for sklearn models

In [17]:
def get_data_matrices(generator):
    x_all = []
    y_all = []
    for x,y in generator:
        x_all.extend(list(x))
        y_all.extend(list(y))

    x_all = np.array(x_all)
    y_all = np.array(y_all)        
    return x_all, y_all

#Make the feature space for this thing.....
def make_features(X):
    s = X.shape
    #              fr    of    mo bp xy
    t = X.reshape((s[0], s[1], 2, 7, 2))
    features = np.zeros((s[0], s[1], 7*7 + 2*21))
    count = 0
    for i in range(7):
        print(f"processing body part {i}")
        for j in range(7):
            if i < j:
                #Intra-distance
                for m in range(2):
                    intra_dist = np.sqrt((t[:,:,m,i,0] - t[:,:,m,j,0])**2 + (t[:,:,m,i,1] - t[:,:,m,j,1])**2)
                    features[:,:,count] = intra_dist
                    count += 1
            #Inter distance
            inter_dist = np.sqrt((t[:,:,0,i,0] - t[:,:,1,j,0])**2 + (t[:,:,0,i,1] - t[:,:,1,j,1])**2)
            features[:,:,count] = inter_dist
            count += 1
            
    return features

#Get rid of some of the extra features
def skip_time_lags(x, skip = 20):
    x_ = x.copy()
    indices = list(range(0, 120, skip))
    x_ = x_[:, indices, :]
    return x_

def compute_differences_time(X):
    periods = [-20, -10, 10, 20]
    data = [X.diff(p) for p in periods] + [X]

def make_data(generator):
    X, y = get_data_matrices(generator)
    X = make_features(X)
    X = skip_time_lags(X)
    X = X.reshape((X.shape[0], -1))
    y = np.argmax(y, axis=-1)
    return X, y

In [18]:
X_st, y_st_train = make_data(trainer.stacking_train_generator)
X_tr, y_train = make_data(trainer.train_generator)
X_va, y_val = make_data(trainer.val_generator)

processing body part 0
processing body part 1
processing body part 2
processing body part 3
processing body part 4
processing body part 5
processing body part 6
processing body part 0
processing body part 1
processing body part 2
processing body part 3
processing body part 4
processing body part 5
processing body part 6
processing body part 0
processing body part 1
processing body part 2
processing body part 3
processing body part 4
processing body part 5
processing body part 6


### Define and train ML models

In [19]:
#Make random forest classifier, with group-level CV
models = {'rf': RandomForestClassifier(n_estimators = 10),
         'lr': sklearn.linear_model.LogisticRegression(max_iter = 10000)}

print('Fitting models:')
for m_name in models:
    print("-", m_name)
    models[m_name].fit(X_tr, y_train)

Fitting models:
- rf
- lr


In [20]:
stacked_features_va = y_val_pred_proba
stacked_features_st = y_stacking_train_pred_proba
stacked_features_tr = y_train_pred_proba

In [42]:
X_tr.shape

(330112, 546)

In [24]:
#Make predictions on validation data and test data
pred_train = {}
pred_val = {}
pred_st = {}

pred_train_proba = {}
pred_st_proba = {}
pred_val_proba = {}

for m in models:
    model = models[m]

    #Compute performance measures
    pred_train[m] = model.predict(X_tr)
    pred_val[m] = model.predict(X_va)
    pred_st[m] = model.predict(X_st)

    pred_train_proba[m] = model.predict_proba(X_tr)
    pred_st_proba[m] = model.predict_proba(X_st)
    pred_val_proba[m] = model.predict_proba(X_va)
    
    stacked_features_va = np.hstack((stacked_features_va, pred_val_proba[m]))
    stacked_features_st = np.hstack((stacked_features_st, pred_st_proba[m]))
    stacked_features_tr = np.hstack((stacked_features_tr, pred_train_proba[m]))

### Look at correlation between predictions

In [50]:
preds = np.array(list(pred_val.values()))
preds = np.vstack((preds, y_val_pred))
np.corrcoef(preds)

array([[1.        , 0.83439257, 0.84502799],
       [0.83439257, 1.        , 0.84974471],
       [0.84502799, 0.84974471, 1.        ]])

### Train final ensemble model

In [25]:
#Train logistic regression (from training data)
final_model = LogisticRegression(max_iter = 10000)

print('Fitting final model')
final_model.fit(stacked_features_st, y_st_train)

Fitting random forest model


LogisticRegression(max_iter=10000)

In [26]:
#Make predictions
y_stacked_st = final_model.predict(stacked_features_st)
y_stacked_va = final_model.predict(stacked_features_va)
y_stacked_tr = final_model.predict(stacked_features_tr)

In [27]:
def compute_metrics(y, pred):
    re = recall_score(y, pred, average = 'macro', labels = [0,1,2])
    pr = precision_score(y, pred, average = 'macro', labels = [0,1,2])
    f1 = f1_score(y, pred, labels = [0,1,2], average = 'macro')
    return [re, pr, f1]

### Performance of stacked model

In [36]:
#Validation data (not used for training underlying model, or stacked ensembl)
print_metrics(compute_metrics(y_val, y_stacked_va))

F1 score: 0.5926282465663969, recall: 0.6675779313384268, precision: 0.613398152670745


In [37]:
#Performance on stacked ensemble training data
print_metrics(compute_metrics(y_st_train, y_stacked_st))

F1 score: 0.6925820704967526, recall: 0.7773195159035139, precision: 0.7193597268770299


In [38]:
#Performance on base model training data
print_metrics(compute_metrics(y_train, y_stacked_tr))

F1 score: 0.29611383459887713, recall: 0.5146810554645531, precision: 0.3370909906467329


### Performance of RF model

In [39]:
print_metrics(compute_metrics(y_val, pred_val['rf']))

F1 score: 0.4594318065089649, recall: 0.5381360495677029, precision: 0.4695136807169081


In [51]:
print_metrics(compute_metrics(y_train, pred_train['rf']))

F1 score: 0.9975561592936438, recall: 0.9991434709616888, precision: 0.9983473245793335


In [52]:
print_metrics(compute_metrics(y_st_train, pred_st['rf']))

F1 score: 0.584133041982892, recall: 0.6059187085329404, precision: 0.5893656806034983


### Performance of NN model

In [40]:
print_metrics(compute_metrics(y_val, y_val_pred))

F1 score: 0.6223424475585833, recall: 0.6684867596984881, precision: 0.6249145701618243


In [57]:
print_metrics(compute_metrics(y_st_train, y_stacking_train_pred)) 

F1 score: 0.6652044833526417, recall: 0.7123358575439322, precision: 0.660325917692368


In [59]:
print_metrics(compute_metrics(y_train, np.argmax(y_train_pred_proba, axis = 1)))

F1 score: 0.1448585742051959, recall: 0.12845538148627347, precision: 0.13602431226185224


## Test trainer prep for submission

In [91]:
#val_predictions = trainer.get_validation_predictions()
print("Getting validation metrics")
val_metrics = trainer.get_validation_metrics()
val_metrics.to_csv(f"{results_dir}/task1_metrics_val.csv", index=False)

if not skip_test_prediction:
    test_results = trainer.get_test_predictions()
    test_proba = trainer.get_test_pred_proba()
    np.save(f"{results_dir}/test_results", test_results)
    np.save(f"{results_dir}/test_results_proba", test_proba)
else:
    test_results = {}
    test_proba = {}

#del trainer # clear ram as the test dataset is large
gc.collect()

Getting validation metrics


100%|██████████| 998/998 [11:21<00:00,  1.46it/s]


3388

In [30]:
self = trainer
all_test_preds = {}
for vkey in self.test_generator.video_keys:
  nframes = self.test_generator.seq_lengths[vkey]
  all_test_preds[vkey] = np.zeros((nframes, 4))

for X, vkey_fi_list in tqdm.tqdm(self.test_generator):
    test_pred = self.model.predict(X)
    #test_pred = np.argmax(test_pred, axis=-1)

    for p, (vkey, fi) in zip(test_pred, vkey_fi_list):
        all_test_preds[vkey][fi, :] = p
#return all_test_preds

100%|██████████| 998/998 [11:46<00:00,  1.41it/s]


## Validate and save the prediction as `npy` 📨

In [ ]:
validate_submission(submission, sample_submission)

In [24]:
np.save("submission.npy", submission)

## Submit to AIcrowd 🚀

In [25]:
!aicrowd submission create -c mabe-task-1-classical-classification -f submission.npy

submission.npy ━━━━━━━━━━━━━━━━━━━━━━ 100.0% • 32.7/32.7 MB • 3.3 MB/s • 0:00:0000:0100:01
                                                                    ╭─────────────────────────╮                                                                    
                                                                    │ Successfully submitted! │                                                                    
                                                                    ╰─────────────────────────╯                                                                    
                                                                          Important links                                                                          
┌──────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/multi-agent-behavior-representati